In [29]:
# Run this to ensure TensorFlow 2.x is used
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

In [30]:
import json
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

In [31]:
vocab_size = 10000 #maximum number of tokens
embedding_dim = 16 #used in the model
max_length = 100
trunc_type='post' 
padding_type='post'
oov_tok = "????"
training_size = 20000

In [32]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json -O sarcasm.json

--2021-05-30 18:47:55--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json
Resolving storage.googleapis.com (storage.googleapis.com)... 216.58.210.80, 142.250.180.48, 142.250.185.48, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|216.58.210.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5643545 (5.4M) [application/json]
Saving to: 'sarcasm.json'

     0K .......... .......... .......... .......... ..........  0% 1.30M 4s
    50K .......... .......... .......... .......... ..........  1% 2.87M 3s
   100K .......... .......... .......... .......... ..........  2% 4.88M 2s
   150K .......... .......... .......... .......... ..........  3% 5.87M 2s
   200K .......... .......... .......... .......... ..........  4% 1.19M 2s
   250K .......... .......... .......... .......... ..........  5% 1.15M 3s
   300K .......... .......... .......... .......... ..........  6% 1.19M 3s
   350K .......... .......... .......... ..

In [33]:
with open("sarcasm.json", 'r') as f:
    datastore = json.load(f)

sentences = []
labels = []

for item in datastore:
    sentences.append(item['headline'])
    labels.append(item['is_sarcastic'])

In [34]:
training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]
training_labels = labels[0:training_size]
testing_labels = labels[training_size:]

In [40]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok)
tokenizer.fit_on_texts(training_sentences) # This assigns numbers to each word

word_index = tokenizer.word_index # This stores the assigned number-word pair in a dictionary

# Converting Training Data to sequences and then padding them
training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_sequences_padded = pad_sequences(training_sequences)

# Converting Testing Data to sequences and then padding them
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_sequences_padded = pad_sequences(testing_sequences)

# Information about training data
print("\nPadded Sequences:")
print(padded)
print("\nShape:", padded.shape)
print("\nThe Maximum length is: ", max(len(i) for i in padded))

# Information about testing data
print("\nPadded Sequences:")
print(padded)
print("\nShape:", padded.shape)
print("\nThe Maximum length is: ", max(len(i) for i in padded))


Padded Sequences:
[[   0    0    0 ...    6 2614 8863]
 [   0    0    0 ...  251    9  889]
 [   0    0    0 ...   45    2 8864]
 ...
 [   0    0    0 ...  417 2231  139]
 [   0    0    0 ...    1  918 8166]
 [   0    0    0 ...    1    1    1]]

Shape: (20000, 40)

The Maximum length is:  40


In [38]:
training_sequences_padded = np.array(training_sequences_padded)
training_labels = np.array(training_labels)
testing_sequences_padded = np.array(testing_sequences_padded)
testing_labels = np.array(testing_labels)